# SF-DAT-21 | Unit Project 3

In this project, you will perform a logistic regression on the admissions data we've been working with in Unit Projects 1 and 2.

In [114]:
import pandas as pd
import numpy as np
import matplotlib.pyplot as plt
import statsmodels.api as sm
import pylab as pl
from sklearn import linear_model
%matplotlib inline

In [2]:
df_raw = pd.read_csv("../../dataset/admissions.csv")
df = df_raw.dropna()
print df.head()

   admit  gre   gpa  prestige
0      0  380  3.61         3
1      1  660  3.67         3
2      1  800  4.00         1
3      1  640  3.19         4
4      0  520  2.93         4


## Part 1. Frequency Tables

#### Question 1. Let's create a frequency table of our variables.

In [35]:
# frequency table for prestige and whether or not someone was admitted
cross_tab = pd.crosstab(index = df['prestige'],
                       columns=df['admit'])
cross_tab

admit,0,1
prestige,,
1,28,33
2,95,53
3,93,28
4,55,12


## Part 2. Return of dummy variables

#### Question 2.1. Create class or dummy variables for prestige.

In [101]:
dummy_ranks = pd.get_dummies(df.prestige, prefix = 'prestige')
dummy_ranks

,prestige_1.0,prestige_2.0,prestige_3.0,prestige_4.0
0,0,0,1,0
1,0,0,1,0
2,1,0,0,0
3,0,0,0,1
4,0,0,0,1
5,0,1,0,0
6,1,0,0,0
7,0,1,0,0
8,0,0,1,0
9,0,1,0,0


In [42]:
df.head(10)

,admit,gre,gpa,prestige,prestige_1.0,prestige_2.0,prestige_3.0,prestige_4.0
0,0,380,3.61,3,0,0,1,0
1,1,660,3.67,3,0,0,1,0
2,1,800,4.00,1,1,0,0,0
3,1,640,3.19,4,0,0,0,1
4,0,520,2.93,4,0,0,0,1
5,1,760,3.00,2,0,1,0,0
6,1,560,2.98,1,1,0,0,0
7,0,400,3.08,2,0,1,0,0
8,1,540,3.39,3,0,0,1,0
9,0,700,3.92,2,0,1,0,0


#### Question 2.2. When modeling our class variables, how many do we need?

Answer: We need 4 to account for the 4 distinct prestige levels that can be classified. 

## Part 3. Hand calculating odds ratios

Develop your intuition about expected outcomes by hand calculating odds ratios.

In [104]:
cols_to_keep = ['admit', 'gre', 'gpa']
data = df[cols_to_keep].join(dummy_ranks.ix[:, 'prestige_1':])
print data.head()

   admit  gre   gpa  prestige_1.0  prestige_2.0  prestige_3.0  prestige_4.0
0      0  380  3.61             0             0             1             0
1      1  660  3.67             0             0             1             0
2      1  800  4.00             1             0             0             0
3      1  640  3.19             0             0             0             1
4      0  520  2.93             0             0             0             1


In [105]:
# crosstab prestige 1 admission
# frequency table cutting prestige and whether or not someone was admitted

prestige1_xtab = pd.crosstab(index = df['prestige_1.0']
                                 ,
                                columns = df['admit'])

prestige1_xtab

admit,0,1
prestige_1.0,,
0,243,93
1,28,33


#### Question 3.1. Use the cross tab above to calculate the odds of being admitted to grad school if you attended a #1 ranked college.

In [94]:
admit_pg1 = 33.0/93.0

#### Question 3.2. Now calculate the odds of admission if you did not attend a #1 ranked college.

In [95]:
no_admit_pg1 = 28.0/243.0

#### Question 3.3. Calculate the odds ratio.

In [96]:
admit_pg1 / no_admit_pg1

3.0794930875576036

#### Question 3.4. Write this finding in a sentenance:

Answer: The odds of admission to a graduate school if you attend a #1 ranked college is 3x higher than not having attended a #1 ranked college. 

#### Question 3.5. Print the cross tab for prestige_4.

In [97]:
prestige4_xtab = pd.crosstab(index = df['prestige_4.0'],
                            columns = df['admit'])
prestige4_xtab

admit,0,1
prestige_4.0,,
0,216,114
1,55,12


#### Question 3.6. Calculate the OR.

In [98]:
admit_pg4 = 12.0 / 114.0
no_admit_pg4 = 55.0/216.0
OR_pg4 = admit_pg4 / no_admit_pg4
OR_pg4

0.4133971291866028

#### Question 3.7. Write this finding in a sentence.

Answer: You are 41% less likely to get admitted into a graduate school if you attened a prestige 4 school than if you attended the other schools. 

## Part 4. Analysis

In [117]:
# create a clean data frame for the regression
cols_to_keep = ['admit', 'gre', 'gpa']
data = df[cols_to_keep].join(dummy_ranks.ix[:, 'prestige_2':])
print data.head()

   admit  gre   gpa  prestige_2.0  prestige_3.0  prestige_4.0
0      0  380  3.61             0             1             0
1      1  660  3.67             0             1             0
2      1  800  4.00             0             0             0
3      1  640  3.19             0             0             1
4      0  520  2.93             0             0             1


We're going to add a constant term for our Logistic Regression.  The statsmodels function we're going to be using requires that intercepts/constants are specified explicitly.

In [118]:
# manually add the intercept
data['intercept'] = 1.0

#### Question 4.1. Set the covariates to a variable called train_cols.

In [119]:
train_cols = data.columns[1:]

#### Question 4.2. Fit the model.

In [121]:
logit = sm.Logit(data['admit'], data[train_cols])
result = logit.fit()

Optimization terminated successfully.
         Current function value: 0.573854
         Iterations 6


#### Question 4.3. Print the summary results.

In [123]:
print result.summary()

                           Logit Regression Results                           
Dep. Variable:                  admit   No. Observations:                  397
Model:                          Logit   Df Residuals:                      391
Method:                           MLE   Df Model:                            5
Date:                Tue, 29 Mar 2016   Pseudo R-squ.:                 0.08166
Time:                        17:44:47   Log-Likelihood:                -227.82
converged:                       True   LL-Null:                       -248.08
                                        LLR p-value:                 1.176e-07
                   coef    std err          z      P>|z|      [95.0% Conf. Int.]
--------------------------------------------------------------------------------
gre              0.0022      0.001      2.028      0.043      7.44e-05     0.004
gpa              0.7793      0.333      2.344      0.019         0.128     1.431
prestige_2.0    -0.6801      0.317     -2.14

#### Question 4.4. Calculate the odds ratios of the coeffincients and their 95% CI intervals

hint 1: np.exp(X)

hint 2: conf['OR'] = params

        conf.columns = ['2.5%', '97.5%', 'OR']

In [124]:
print np.exp(result.params)

gre             1.002221
gpa             2.180027
prestige_2.0    0.506548
prestige_3.0    0.262192
prestige_4.0    0.211525
intercept       0.020716
dtype: float64


#### Question 4.5. Interpret the OR of Prestige_2.

Answer: You are 50% less likely to be admitted to graduate school if you attended a school of prestige 2.0 than someone who attended prestige school of 1.0

#### Question 4.6. Interpret the OR of GPA.

Answer: You are about twice as likely to be accepted into graduate school if you have a higher gpa than somone who attended a prestige 1 school 

## Part 5: Predicted probablities


As a way of evaluating our classifier, we're going to recreate the dataset with every logical combination of input values.  This will allow us to see how the predicted probability of admission increases/decreases across different variables.  First we're going to generate the combinations using a helper function called cartesian (above).

We're going to use np.linspace to create a range of values for "gre" and "gpa".  This creates a range of linearly spaced values from a specified min and maximum value--in our case just the min/max observed values.

In [143]:
def cartesian(arrays, out=None):
    """
    Generate a cartesian product of input arrays.
    Parameters
    ----------
    arrays : list of array-like
        1-D arrays to form the cartesian product of.
    out : ndarray
        Array to place the cartesian product in.
    Returns
    -------
    out : ndarray
        2-D array of shape (M, len(arrays)) containing cartesian products
        formed of input arrays.
    Examples
    --------
    >>> cartesian(([1, 2, 3], [4, 5], [6, 7]))
    array([[1, 4, 6],
           [1, 4, 7],
           [1, 5, 6],
           [1, 5, 7],
           [2, 4, 6],
           [2, 4, 7],
           [2, 5, 6],
           [2, 5, 7],
           [3, 4, 6],
           [3, 4, 7],
           [3, 5, 6],
           [3, 5, 7]])
    """

    arrays = [np.asarray(x) for x in arrays]
    dtype = arrays[0].dtype

    n = np.prod([x.size for x in arrays])
    if out is None:
        out = np.zeros([n, len(arrays)], dtype=dtype)

    m = n / arrays[0].size
    out[:,0] = np.repeat(arrays[0], m)
    if arrays[1:]:
        cartesian(arrays[1:], out=out[0:m,1:])
        for j in xrange(1, arrays[0].size):
            out[j*m:(j+1)*m,1:] = out[0:m,1:]
    return out

In [144]:
# instead of generating all possible values of GRE and GPA, we're going
# to use an evenly spaced range of 10 values from the min to the max
gres = np.linspace(data['gre'].min(), data['gre'].max(), 10)

print gres
# array([ 220.        ,  284.44444444,  348.88888889,  413.33333333,
#         477.77777778,  542.22222222,  606.66666667,  671.11111111,
#         735.55555556,  800.        ])

gpas = np.linspace(data['gpa'].min(), data['gpa'].max(), 10)

print gpas
# array([ 2.26      ,  2.45333333,  2.64666667,  2.84      ,  3.03333333,
#         3.22666667,  3.42      ,  3.61333333,  3.80666667,  4.        ])

# enumerate all possibilities
combos = pd.DataFrame(cartesian([gres, gpas, [1, 2, 3, 4], [1.]]))

[ 220.          284.44444444  348.88888889  413.33333333  477.77777778
  542.22222222  606.66666667  671.11111111  735.55555556  800.        ]
[ 2.26        2.45333333  2.64666667  2.84        3.03333333  3.22666667
  3.42        3.61333333  3.80666667  4.        ]


#### Question 5.1. Recreate the dummy variables.

In [145]:
# recreate the dummy variables
combos.columns = ['gre', 'gpa', 'prestige', 'intercept']
dummy_ranks = pd.get_dummies(combos['prestige'], prefix='prestige')
dummy_ranks.columns = ['prestige_1', 'prestige_2', 'prestige_3', 'prestige_4']

# keep only what we need for making predictions
cols_to_keep = ['gre', 'gpa', 'prestige', 'intercept']
combos = combos[cols_to_keep].join(dummy_ranks.ix[:, 'prestige_2':])

#### Question 5.2. Make predictions on the enumerated dataset.

In [146]:
train_cols = combos.columns[1:]

combos['admit_pred'] = result.predict(combos[train_cols])

In [142]:
print combos.tail()

     gre       gpa  prestige  intercept  prestige_2  prestige_3  prestige_4  \
395  800  3.806667         4          1           0           0           1   
396  800  4.000000         1          1           0           0           0   
397  800  4.000000         2          1           1           0           0   
398  800  4.000000         3          1           0           1           0   
399  800  4.000000         4          1           0           0           1   

     admit_pred  
395    0.192912  
396    0.526992  
397    0.389060  
398    0.528304  
399    0.192979  


#### Question 5.3. Interpret findings for the last 4 observations.

Answer: 
observation #396 - 48% less likely to be admitted to graduate school than the control group.
observation #397 - 62% less likely to be admitted to graduate school than the control group.
observation #398 - 48% less likely to be admitted to graduate school than the control group. 
observation #399 - 81% less likely to be admitted to graduate school than the control group. 

The control group is those who attended a prestige 1 school. 

## Bonus

Plot the probability of being admitted into graduate school, stratified by GPA and GRE score.